In [1]:
#read pdf
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("js_pdf.pdf")

documents = loader.load()

In [3]:
#use text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(texts)} chunks")

Split 29 documents into 41 chunks


In [4]:
documents[:5]

[Document(page_content='\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nBeginner’s Essential\n \nJavascript Cheat Sheet\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nThe language of the web.\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n \n \n ', metadata={'source': 'js_pdf.pdf', 'page': 0}),
 Document(page_content='Table of Contents\n\xa0\n \nJavascript Basics\n \n2\n \nVariables\n \n2\n \nArrays\n \n3\n \nOperators\n \n4\n \nFunctions\n \n5\n \nLoops\n \n7\n \nIf - Else Statements\n \n7\n \nStrings\n \n7\n \nRegular Expressions\n \n9\n \nNumbers and Math\n \n10\n \nDealing with Dates\n \n12\n \nDOM Node\n \n14\n \nWorking with the Browser\n \n18\n \nEvents\n \n21\n \nErrors\n \n27\n \n \n \n\xa0\nWebsiteSetup.org - Beginner’s Javascript Cheat Sheet                                                                                      \u200b\n  \u200b\n1\n ', metadata={'source': 'js_pdf.pdf', 'page': 1}),
 Document(page_c

In [8]:
#code for embedding
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = OllamaEmbeddings(model='mxbai-embed-large')

db = Chroma.from_documents(texts[:5], embeddings)


In [9]:
from langchain_community.llms import ollama
llm = ollama.Ollama(model="gemma:2b",temperature=1)


In [23]:
#creating chat prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
you are a helpful assistant who will give me answer to the following questions based on the provided context
<context>
{context}
</context>

question:{input}
"""
)

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8CED3D2E0>)

In [26]:
from langchain.chains import create_retrieval_chain
retriveal_chain = create_retrieval_chain(retriver,document_chain)

In [34]:
response = retriveal_chain.invoke({"input":"what is the title of the text?"})
# response.

In [35]:
response['answer']

"The provided text is a Beginner's Essential Javascript Cheat Sheet."

In [36]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("sample.txt")

text = loader.load()
bd = Chroma.from_documents(text,embeddings)
text_retrive = db.as_retriever()


In [38]:
text_retriver_chain = create_retrieval_chain(retriver,document_chain)#you shoulld create a new db for this process